### Load Dependencies

In [1]:
import requests
import json
from datetime import datetime, timedelta
import os
from unittest.mock import patch
import boto3

### Some constants

In [2]:
# Define locations
locations = {
    "us-west-2": (43.8041334, -120.5542012),
    "us-west-1": (38.8375215, -120.8958242),
}

start_date = datetime(2023, 6, 1, 0, 0, 0)

end_date = datetime(2023, 6, 30, 0, 0, 0)

### Load Carbon Data

In [16]:
# Function to query the API
def query_api(start, end, lat, lon, token):
    url = f"https://api-access.electricitymaps.com/free-tier/carbon-intensity/past-range?lat={lat}&lon={lon}&start={start}&end={end}"
    headers = {"auth-token": token}
    response = requests.get(url, headers=headers, timeout=10)
    if response.status_code == 200:
        return response.json()["data"]
    else:
        print(f"Error querying API: {response.status_code}")
        return []

def run():
    for location, (latitude, longitude) in locations.items():
        current_start = start_date - timedelta(days=7) # Start 7 days before the start date
        combined_data = []

        while current_start < end_date:
            current_end = current_start + timedelta(days=10)
            if current_end > end_date:
                current_end = end_date
            data = query_api(
                current_start, current_end, latitude, longitude, os.environ.get("ELECTRICITY_MAPS_AUTH_TOKEN")
            )
            combined_data.extend(data)
            current_start = current_end + timedelta(days=1)

        # Save the combined data to a file
        output_file = f"./data/carbon/{location}_carbon_data.json"
        with open(output_file, "w") as f:
            json.dump(combined_data, f)

        print(f"Data combined and saved to {output_file}")


if __name__ == "__main__":
    run()

Data combined and saved to ./data/carbon/us-west-2_carbon_data.json
Data combined and saved to ./data/carbon/us-west-1_carbon_data.json


### Create Carbon Collector Logs

In [18]:
import json
import math
import os
from datetime import datetime, timedelta
from collections import defaultdict

def calculate_distance(lat1, lon1, lat2, lon2):
    r = 6371.0  # Earth radius in kilometers
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2)**2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return r * c

def process_and_store_carbon_data_for_regions(input_file, output_folder, regions_info, current_region):
    os.makedirs(output_folder, exist_ok=True)  # Ensure output directory exists
    
    with open(input_file, 'r') as file:
        data = json.load(file)
    
    # For each day between start_date and end_date
    for day in range((end_date - start_date).days + 1):
        current_date = start_date + timedelta(days=day)

        # Get all the data for the previous 7 days
        previous_7_days_data = [entry for entry in data if datetime.strptime(entry['datetime'], '%Y-%m-%dT%H:%M:%S.%fZ').date() >= (current_date - timedelta(days=7)).date() and datetime.strptime(entry['datetime'], '%Y-%m-%dT%H:%M:%S.%fZ').date() < current_date.date()]

        # Calculate the average carbon intensity for the previous 7 days
        if not previous_7_days_data:
            continue
        overall_sum = sum(item['carbonIntensity'] for item in previous_7_days_data)
        overall_avg = overall_sum / len(previous_7_days_data)

        # Calculate the average carbon intensity for each hour of the day
        hourly_averages = defaultdict(list)
        for item in previous_7_days_data:
            hour = datetime.strptime(item['datetime'], '%Y-%m-%dT%H:%M:%S.%fZ').hour
            hourly_averages[hour].append(item['carbonIntensity'])

        hourly_avg = {hour: sum(values) / len(values) for hour, values in hourly_averages.items()}

        # Calculate the distances between the regions
        transmission_distances = {
            f"aws:{region_key}": calculate_distance(regions_info[current_region][0], regions_info[current_region][1], regions_info[region_key][0], regions_info[region_key][1]) for region_key in regions_info
        }
    
        # Assemble the result dictionary
        result_dict = {
            "averages": {
                "overall": {"carbon_intensity": overall_avg},
                **{str(hour): {"carbon_intensity": avg} for hour, avg in hourly_avg.items()}
            },
            "units": "gCO2eq/kWh",
            "transmission_distances": transmission_distances
        }

        # Store the result
        day_folder = os.path.join(output_folder, current_date.strftime('%Y-%m-%d'))
        os.makedirs(day_folder, exist_ok=True)
        with open(os.path.join(day_folder, 'data.json'), 'w') as outfile:
            json.dump(result_dict, outfile, indent=4)

for region in locations:
    process_and_store_carbon_data_for_regions(f"./data/carbon/{region}_carbon_data.json", f"./data/collected_carbon/{region}", locations, region)

### Run the Deployment Algorithm

We need to run the deployment algorithm for the days between the start and end date. We will run it every day and store the results locally in files.
Additionally, we need to provide the data collected by a specific workflow as the input to the workflow loader.

In [3]:
from multi_x_serverless.common.constants import GLOBAL_SYSTEM_REGION, DEPLOYMENT_OPTIMIZATION_MONITOR_RESOURCE_TABLE
from multi_x_serverless.routing.workflow_config import WorkflowConfig
from multi_x_serverless.routing.deployment_algorithms.stochastic_heuristic_deployment_algorithm import StochasticHeuristicDeploymentAlgorithm
from multi_x_serverless.routing.deployment_input.components.loaders.carbon_loader import CarbonLoader
from multi_x_serverless.routing.deployment_input.components.loaders.region_viability_loader import RegionViabilityLoader
from multi_x_serverless.routing.deployment_algorithms.deployment_algorithm import DeploymentAlgorithm

# First define the involved benchmarks, these should already be deployed and have workflow data
benchmarks = ["dna_visualization-0.0.1"]

constraint_configurations = {
    "no_constraints": {
        "constraints": {
            "hard_resource_constraints": {"cost": None, "runtime": None, "carbon": None},
            "soft_resource_constraints": {"cost": None, "runtime": None, "carbon": None},
            "priority_order": ["carbon", "cost", "runtime"],
        }
    },
    "five_percent_runtime_constraint": {
        "constraints": {
            "hard_resource_constraints": {"cost": None, "runtime": {"type": "relative", "value": 105}, "carbon": None},
            "soft_resource_constraints": {"cost": None, "runtime": None, "carbon": None},
            "priority_order": ["carbon", "cost", "runtime"],
        }
    },
}

dynamodb_client = boto3.client('dynamodb', region_name=GLOBAL_SYSTEM_REGION)
for benchmark in benchmarks:
    response = dynamodb_client.get_item(TableName=DEPLOYMENT_OPTIMIZATION_MONITOR_RESOURCE_TABLE, Key={"key": {"S": benchmark}})
    item = response.get("Item")
    workflow_config_from_table = item["value"]["S"]

    workflow_config_dict = json.loads(json.loads(workflow_config_from_table).get("workflow_config"))
    for configuration_name, configuration in constraint_configurations.items():
        workflow_config_dict["constraints"] = configuration["constraints"]
        workflow_config = WorkflowConfig(workflow_config_dict)

        for day in range((end_date - start_date).days + 1):
            current_date = start_date + timedelta(days=day)

            def mock_carbon_loader_setup(self, available_regions: set[str]):
                real_available_regions = list(locations.keys())
                loaded_carbon_data = {}
                for region in real_available_regions:
                    carbon_data_path = f'./data/collected_carbon/{region}/{current_date.strftime("%Y-%m-%d")}/data.json'
                    with open(carbon_data_path, 'r') as file:
                        loaded_carbon_data[f'aws:{region}'] = json.load(file)
                self._carbon_data = loaded_carbon_data

            def mock_region_viability_setup(self):
                self._available_regions = [f'aws:{region}' for region in locations.keys()]

            def mock_upload_result(self, result: dict):
                result_path = f'./data/deployment_results/{benchmark}/{configuration_name}/'
                os.makedirs(result_path, exist_ok=True)
                with open(os.path.join(result_path, f'{current_date.strftime("%Y-%m-%d")}.json'), 'w') as file:
                    json.dump(result, file, indent=4)

            with patch.object(CarbonLoader, 'setup', new=mock_carbon_loader_setup), patch.object(RegionViabilityLoader, 'setup', new=mock_region_viability_setup), patch.object(DeploymentAlgorithm, '_upload_result', new=mock_upload_result):
                algorithm = StochasticHeuristicDeploymentAlgorithm(workflow_config)
                algorithm.run([f"{i}" for i in range(24)])